In [1]:
import torch
import pyro

pyro.set_rng_seed(101)

# Primitive Stochastic Functions

Primitive Stochastic Functionsの意味がわからない

In [2]:
loc = 0
scale = 1
normal = torch.distributions.Normal(loc, scale)
x = normal.rsample()
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-1.3905)
log prob tensor(-1.8857)


## A Simple Model
ベルヌーイと正規分布からなる混合モデル

In [6]:
def weather():
    # 30% cloudy
    # NOTE: sample(), rsample()の違いは？
    # rsample()だと微分可能になる
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = "cloudy" if cloudy.item() == 1 else "sunny"
    mean_temp = {"cloudy": 55.0, "sunny": 75.0}[cloudy]
    scale_temp = {"cloudy": 10.0, "sunny": 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

In [29]:
weather()

('sunny', 73.0761947631836)

In [7]:
# The pyro.sample Primitive
x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
print(x)

tensor(-0.3204)


 NOTE: pyro.sampleを使おうねという話
 -> 一つの変数に名前を持ったランダムな変数を複数個持たせられるので
 -> つまり、混合確率分布を表現できる

In [8]:
def weather():
    cloudy = pyro.sample("cloudy", pyro.distributions.Bernoulli(0.3))
    cloudy = "cloudy" if cloudy.item() == 1 else "sunny"
    mean_temp = {"cloudy": 55.0, "sunny": 75.0}[cloudy]
    scale_temp = {"cloudy": 10.0, "sunny": 15.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

for _ in range(3):
    print(weather())

('sunny', 86.06607818603516)
('sunny', 84.56695556640625)
('sunny', 54.924041748046875)


## Universality: Stochastic Recursion, Higher-order Stochastic Functions, and Random Control Flow

In [9]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream

In [13]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.5))

1


In [15]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.))

tensor(0.0930)
